## This code codebook serves multiple purposes

- Join downstream outcomes from the post-session survey, premature departure table and attention score data
- Comparisons between non-concurrent sessions with concurrent sessions
- Comparisons within groups of concurrent sessions
- Explore optimal attention score on downstream outcomes

@author: Yucan Xu (chicoxu@connect.hku.hk)


In [ ]:
import pandas as pd
import numpy as np
import glob,os,re,string
import pytz
import datetime
import pymongo
import re
import pingouin as pg
from pingouin import ttest
import researchpy as rp
from scipy.stats import pearsonr
from datetime import timedelta
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
from scipy.stats import chi2_contingency
from itertools import combinations
from statsmodels.sandbox.stats.multicomp import multipletests
from pingouin import kruskal, read_dataset

## read attention score with premature departure data

In [ ]:
concurrent = pd.read_csv('attention_score.csv')

In [ ]:
concurrent

In [ ]:
premature_departure = pd.read_csv('premature_depature_detected_20-21.csv')

In [ ]:
premature_departure.columns

In [ ]:
premature_departure.startDateTime.min()

In [ ]:
premature_departure.startDateTime.max()

## join table that include counselors' role and dates of visits from users

In [ ]:
case_roles = pd.read_csv('../volunteer_training/cases_by_roles.csv')

In [ ]:
case_roles.columns

In [ ]:
concurrent = concurrent.join(case_roles[['uuid','role']].set_index('uuid'),
                            on='conversationId',how='left')

In [ ]:
concurrent['role'].value_counts()

In [ ]:
role_tag = {
    'DUTY_OFFICER':'Paid_staff',
    'SOCIAL_WORKER':'Paid_staff',
    'Relief_Worker':'Paid_staff'
}

In [ ]:
concurrent['role'] = concurrent['role'].replace(role_tag)

In [ ]:
concurrent['role'].value_counts()

In [ ]:
concurrent['visit_hour'] = concurrent['startDateTime'].apply(
    lambda x:x.split(' ')[1].split(':')[0])

In [ ]:
hours = concurrent['visit_hour'].value_counts().reset_index()

In [ ]:
hours = hours.rename(columns={'index':'visiting_hour'})

In [ ]:
hours['visiting_hour'] = hours['visiting_hour'].replace('00','24')

In [ ]:
hours = hours.sort_values(by='visiting_hour')

In [ ]:
hours.plot(x = 'visiting_hour',y='visit_hour',  ylabel='case_count', 
           figsize = (10,6),kind='bar', legend=False)

In [ ]:
concurrent['visit_hour'] = concurrent['visit_hour'].replace('00','24')

In [ ]:
concurrent['visit_hour'] = concurrent['visit_hour'].astype(int)

In [ ]:
concurrent['visting_period'] = concurrent['visit_hour'].apply(lambda x:'1' if x<=8 else('3' if x>=17 else '2'))

In [ ]:
concurrent['visting_period'].value_counts()

In [ ]:
data_join = concurrent.join(
    premature_departure[['conversationId','premature_departure_predicted']].set_index('conversationId'),
on='conversationId',how='left')

## Seperate concurrent and non-concurrent (single) session

In [ ]:
data_join['single'] = np.where(data_join['max_concurrent_group']>=2,
                              1,0)

In [ ]:
data_join.groupby('single')['premature_departure_predicted'].value_counts()

In [ ]:
data_join

In [ ]:
data_multiple_chats = data_join[data_join['single']!=0].copy()

In [ ]:
exog = sm.add_constant(data_multiple_chats[['attention_score']])

In [ ]:
endog = data_multiple_chats[['premature_departure_predicted']]

In [ ]:
logistic_model = sm.GLM(endog, exog,
                        family=sm.families.Binomial())

In [ ]:
results = logistic_model.fit()

In [ ]:
print(results.summary())

In [ ]:
params = results.params
conf = results.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
np.exp(conf)

## Join post_session survey

In [ ]:
ps = pd.read_excel('../openup_survey/post_chat_survey_2112.xlsx')

In [ ]:
ps.columns

In [ ]:
data_join2 = data_join.join(
    ps[['個案編號','1. 比起對話前，你現在的感覺如何?.1','3. 你認為本對話服務對你有多大幫助？[1至5選一]',
       '6. 使用本服務後，是否能夠否鼓勵你日後在有需要時尋求協助?','7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？']].set_index('個案編號'),on='conversationId',
    how='left')

In [ ]:
data_join2.columns

In [ ]:
data_join2['filling'] = (data_join2['1. 比起對話前，你現在的感覺如何?.1'].isna()) & (data_join2['3. 你認為本對話服務對你有多大幫助？[1至5選一]'].isna()) & (data_join2['6. 使用本服務後，是否能夠否鼓勵你日後在有需要時尋求協助?'].isna()) & (data_join2['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'].isna())

In [ ]:
data_join2['filling'].value_counts()

In [ ]:
data_join2['filling'] = data_join2['filling'].replace(True,'0')

In [ ]:
data_join2['filling'] = data_join2['filling'].replace(False,'1')

In [ ]:
data_join2.groupby('single')['filling'].value_counts()

In [ ]:
data_join2.groupby('single')['1. 比起對話前，你現在的感覺如何?.1'].value_counts()

### Clean question answers 

In [ ]:
q1_place_tag = {'好咗好多':5,
               '好咗一啲':4,
               '沒有變化':3,
               '差咗一啲':2,
               '差咗好多':1,
               '好了很多':5,
               '好了一些':4,
               '沒有变化':3,
               '差了一些':2,
               '差了很多':1,
               'I feel much better':5,
               'I feel a bit better':4,
               'I feel no change':3,
               'I feel a bit worse than before':2,
               'I feel much worse than before':1}

In [ ]:
data_join2['1. 比起對話前，你現在的感覺如何?.1'] = data_join2['1. 比起對話前，你現在的感覺如何?.1'].replace(q1_place_tag)

## Comparisons on downstream outcomes between concurrent sessions and non-concurrent sessions

### Q1-part A ttest

In [ ]:
x = data_join2[data_join2['single']==0]['1. 比起對話前，你現在的感覺如何?.1']

In [ ]:
y = data_join2[data_join2['single']==1]['1. 比起對話前，你現在的感覺如何?.1']

In [ ]:
rp.ttest(group1= x, group1_name= "Single",
         group2= y, group2_name= "Concurrent")

### Q1-Part B linear regression

In [ ]:
data = data_join2[data_join2['1. 比起對話前，你現在的感覺如何?.1'].notna()].copy()

In [ ]:
data  = data.rename(columns={'1. 比起對話前，你現在的感覺如何?.1':'q1'})

In [ ]:
lmodel1 = smf.ols(formula='q1 ~ attention_score', data=data).fit() 
print(lmodel1.summary())

### Q3 - part A

In [ ]:
x = data_join2[data_join2['single']==0]['3. 你認為本對話服務對你有多大幫助？[1至5選一]']
y = data_join2[data_join2['single']==1]['3. 你認為本對話服務對你有多大幫助？[1至5選一]']

In [ ]:
rp.ttest(group1= x, group1_name= "Single",
         group2= y, group2_name= "Concurrent")

### Q3 Part B

In [ ]:
data = data_join2[data_join2['3. 你認為本對話服務對你有多大幫助？[1至5選一]'].notna()].copy()

In [ ]:
data  = data.rename(columns={'3. 你認為本對話服務對你有多大幫助？[1至5選一]':'q3'})

In [ ]:
lmodel3 = smf.ols(formula='q3 ~ attention_score', data=data).fit() 
print(lmodel3.summary())

### Q7 part A

In [ ]:
q7_tag = {
    'Yes':'會',
    'No':'不會',
    'Not sure':'不確定',
    '不确定':'不確定',
    '不会':'不會',
    '会':'會'
}

In [ ]:
data_join2['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'] = data_join2['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'].replace(
q7_tag)

In [ ]:
data_join2.groupby('single')['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'].value_counts()

In [ ]:
data_join2['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'] = data_join2['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'].replace('不確定','不會').copy()

In [ ]:
rp.crosstab(data_join2['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'],
            data_join2['single'], test= "chi-square")

### Q7 part B

In [ ]:
q7_tag2 = {'不會':0,
          '會':1}

In [ ]:
data = data_join2[data_join2['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'].notna()].copy()

In [ ]:
data = data.rename(columns={'7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？':'Q7'})

In [ ]:
data['Q7'] = data['Q7'].replace(q7_tag2)

In [ ]:
exog = sm.add_constant(data[['attention_score']])

In [ ]:
endog = data[['Q7']]

In [ ]:
logistic_model3 = sm.GLM(endog, exog,
                        family=sm.families.Binomial())

In [ ]:
results = logistic_model3.fit()

In [ ]:
print(results.summary())

In [ ]:
params = results.params
conf = results.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
np.exp(conf)

### Premature departure

In [ ]:
rp.crosstab(data_join2['premature_departure_predicted'],
            data_join2['single'], test= "chi-square")

In [ ]:
data_join2['single'].value_counts()

### Chat time

In [ ]:
from pytimeparse import parse

In [ ]:
data_join2['chat_duration_cast'] = data_join2['chat_duration'].apply(
    lambda x:parse(x)/60)

In [ ]:
x = data_join2[data_join2['single']==0]['chat_duration_cast']
y = data_join2[data_join2['single']==1]['chat_duration_cast']

In [ ]:
rp.ttest(group1= x, group1_name= "Single",
         group2= y, group2_name= "Concurrent")

In [ ]:
lmodel4 = smf.ols(formula='chat_duration_cast ~ attention_score', data=data_join2).fit() 
print(lmodel4.summary())

In [ ]:
data_join4 = data_join2[data_join2['chat_duration_cast']<=360].copy()

In [ ]:
data_join4.groupby('single')['chat_duration_cast'].describe()

## Identify optimal attention score

In [ ]:
data_join2.columns

In [ ]:
data_join2['attention_score_smooth'] = data_join2['attention_score'].round(-1)

In [ ]:
accumulative = data_join2.groupby('attention_score_smooth').agg({'premature_departure_predicted':'sum',
                                                         'conversationId':'count',
                                                         '1. 比起對話前，你現在的感覺如何?.1':'sum',
                                                         '3. 你認為本對話服務對你有多大幫助？[1至5選一]':'sum',
                                                         '7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？':'sum'
                                                         }).reset_index()

In [ ]:
accumulative

In [ ]:
accumulative['cn_total'] = accumulative['conversationId'].cumsum()

In [ ]:
accumulative['pd_total'] = accumulative['premature_departure_predicted'].cumsum()

In [ ]:
accumulative['pd_percentage'] = accumulative['pd_total']/accumulative['cn_total']*100

In [ ]:
#accumulative

In [ ]:
accumulative.plot(x='attention_score_smooth', y='pd_percentage', kind='line',
        figsize=(12, 8), legend=False, style="ro-")
plt.title("(a). Accumulative percentage of premature departure on attention score", y=1.01, fontsize=18)
plt.ylabel("Accumulative Percentage of premature departure %", labelpad=15,fontsize=14)
plt.xlabel("Attention score", labelpad=15,fontsize=14);
plt.savefig("percentage_accumulative_pd_s.jpg", dpi=300)

In [ ]:
data = data_join2[data_join2['1. 比起對話前，你現在的感覺如何?.1'].notna()].copy()

## 

In [ ]:
accumulative2 = data.groupby('attention_score_smooth').agg({'1. 比起對話前，你現在的感覺如何?.1':'sum',
                                                    'conversationId':'count'}).reset_index()

In [ ]:
accumulative2

In [ ]:
accumulative2['cn_total'] = accumulative2['conversationId'].cumsum()

In [ ]:
accumulative2['score_total'] = accumulative2['1. 比起對話前，你現在的感覺如何?.1'].cumsum()

In [ ]:
accumulative2['score_mean'] = accumulative2['score_total']/accumulative2['cn_total']

In [ ]:
accumulative2['score_mean'] = accumulative2['score_mean'].round(2)

In [ ]:
accumulative2.plot(x='attention_score_smooth', y='score_mean', kind='line',
        figsize=(12, 8), legend=False, style="ro-")
plt.title('(c). Accumulative mean score of "feeling better after the session" on attention score', y=1.01, fontsize=18)
plt.yticks(np.arange(accumulative2['score_mean'].min(), accumulative2['score_mean'].max(), 0.05))
plt.ylabel('Accumulative mean score of "feeling better after the session"', labelpad=15,fontsize=14)
plt.xlabel("Attention score", labelpad=15,fontsize=14);
plt.savefig("score_q1_s.jpg", dpi=300)

In [ ]:
data = data_join2[data_join2['3. 你認為本對話服務對你有多大幫助？[1至5選一]'].notna()].copy()

In [ ]:
accumulative3 = data.groupby('attention_score_smooth').agg({'3. 你認為本對話服務對你有多大幫助？[1至5選一]':'sum',
                                                    'conversationId':'count'}).reset_index()

In [ ]:
accumulative3['cn_total'] = accumulative3['conversationId'].cumsum()

In [ ]:
accumulative3['score_total'] = accumulative3['3. 你認為本對話服務對你有多大幫助？[1至5選一]'].cumsum()

In [ ]:
accumulative3['score_mean'] = accumulative3['score_total']/accumulative3['cn_total']

In [ ]:
accumulative3['score_mean'] = accumulative3['score_mean'].round(2)

In [ ]:
accumulative3

In [ ]:
accumulative3.plot(x='attention_score_smooth', y='score_mean', kind='line',
        figsize=(12, 8), legend=False, style="ro-")
plt.title('(d). Accumulative mean score of "feeling helpful about the session" on attention score', y=1.01, fontsize=18)
plt.yticks(np.arange(accumulative3['score_mean'].min(), accumulative3['score_mean'].max()+0.05, 0.05))
plt.ylabel('Accumulative mean score of "feeling helpful about the session"', labelpad=15,fontsize=14)
plt.xlabel("Attention score", labelpad=15,fontsize=14);
plt.savefig("score_q3_s.jpg", dpi=300)

In [ ]:
data = data_join2[data_join2['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'].notna()].copy()

In [ ]:
accumulative4 = data.groupby('attention_score_smooth').agg({'7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？':'sum',
                                                    'conversationId':'count'}).reset_index()

In [ ]:
accumulative4['cn_total'] = accumulative4['conversationId'].cumsum()

In [ ]:
accumulative4['score_total'] = accumulative4['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'].cumsum()

In [ ]:
accumulative4['rm_percentage'] = accumulative4['score_total']/accumulative4['cn_total']*100

In [ ]:
accumulative4

In [ ]:
accumulative4.plot(x='attention_score_smooth', y='rm_percentage', kind='line',
        figsize=(12, 8), legend=False, style="ro-")
plt.title("(b). Accumulative percentage of willing to recommend the service on attention score", y=1.01, fontsize=18)
plt.ylabel("Accumulative Percentage of willing to recommend the service %", labelpad=15,fontsize=14)
plt.xlabel("Attention score", labelpad=15,fontsize=14);
plt.savefig("percentage_accumulative_rm_s.jpg", dpi=300)

## Statistical test between concurrent groups

In [ ]:
aov = pg.anova(dv='chat_duration_cast', between='max_concurrent_group2', data=data_join2,
               detailed=True)
aov

In [ ]:
data_join2.pairwise_tukey(dv='chat_duration_cast', between='max_concurrent_group2',effsize='eta-square').round(3)

In [ ]:
data = data_join2[data_join2['1. 比起對話前，你現在的感覺如何?.1'].notna()].copy()

In [ ]:
aov = pg.anova(dv='1. 比起對話前，你現在的感覺如何?.1', between='max_concurrent_group2', data=data,
               detailed=True)
aov

In [ ]:
data.pairwise_tukey(dv='1. 比起對話前，你現在的感覺如何?.1', between='max_concurrent_group2',effsize='eta-square').round(3)

In [ ]:
aov = pg.anova(dv='3. 你認為本對話服務對你有多大幫助？[1至5選一]', between='max_concurrent_group2', data=data_join2,
               detailed=True)
aov

In [ ]:
data_join2.pairwise_tukey(dv='3. 你認為本對話服務對你有多大幫助？[1至5選一]', between='max_concurrent_group2',effsize='eta-square').round(3)

In [ ]:
kruskal(data=data_join2, dv='premature_departure_predicted', between='max_concurrent_group2')

In [ ]:
def get_asterisks_for_pval(p_val):
    """Receives the p-value and returns asterisks string."""
    if p_val > 0.05:
        p_text = "ns"  # above threshold => not significant
    elif p_val < 1e-4:  
        p_text = '****'
    elif p_val < 1e-3:
        p_text = '***'
    elif p_val < 1e-2:
        p_text = '**'
    else:
        p_text = '*'
    
    return p_text

In [ ]:
def chisq_and_posthoc_corrected(df):
    """Receives a dataframe and performs chi2 test and then post hoc.
    Prints the p-values and corrected p-values (after FDR correction)"""
    # start by running chi2 test on the matrix
    chi2, p, dof, ex = chi2_contingency(df, correction=True)
    print(f"Chi2 result of the contingency table: {chi2}, p-value: {p}")
    
    # post-hoc
    all_combinations = list(combinations(df.index, 2))  # gathering all combinations for post-hoc chi2
    p_vals = []
    print("Significance results:")
    for comb in all_combinations:
        new_df = df[(df.index == comb[0]) | (df.index == comb[1])]
        chi2, p, dof, ex = chi2_contingency(new_df, correction=True)
        p_vals.append(p)
        # print(f"For {comb}: {p}")  # uncorrected

    # checking significance
    # correction for multiple testing
    reject_list, corrected_p_vals = multipletests(p_vals, method='fdr_bh')[:2]
    for p_val, corr_p_val, reject, comb in zip(p_vals, corrected_p_vals, reject_list, all_combinations):
        print(f"{comb}: p_value: {p_val:5f}; corrected: {corr_p_val:5f} ({get_asterisks_for_pval(p_val)}) reject: {reject}")

In [ ]:
q1_con = data_join2.groupby('max_concurrent_group2')['premature_departure_predicted'].value_counts().unstack()

In [ ]:
chisq_and_posthoc_corrected(q1_con)

In [ ]:
q7_con = data_join2.groupby('max_concurrent_group2')['7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？'].value_counts().unstack()

In [ ]:
q7_con

In [ ]:
chisq_and_posthoc_corrected(q7_con)